LAB 2 - Milica miskovic

First, let us import all packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.decomposition import PCA


Then we read the data file into data frame

In [3]:
ratings = pd.read_csv('ratings.dat', sep="::", header=None, engine='python')
ratings.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings.drop(['Timestamp'], axis=1, inplace=True)
print(ratings[:5])

   UserID  MovieID  Rating
0       1     1193       5
1       1      661       3
2       1      914       3
3       1     3408       4
4       1     2355       5


1. Create m x u matrix with movies as row and users as column

In [148]:
ratings_pivot = ratings.pivot_table('Rating', index='MovieID', columns='UserID')
print(ratings_pivot.iloc[0:10, 0:10])

UserID    1   2   3   4    5    6    7    8    9    10
MovieID                                               
1        5.0 NaN NaN NaN  NaN  4.0  NaN  4.0  5.0  5.0
2        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  5.0
3        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
4        NaN NaN NaN NaN  NaN  NaN  NaN  3.0  NaN  NaN
5        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
6        NaN NaN NaN NaN  2.0  NaN  4.0  NaN  NaN  NaN
7        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  4.0
8        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
9        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
10       NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN


2. Normalize the matrix (we have to fill NaNs with 0 first, and call it matrix a)

In [135]:
a = ratings_pivot.fillna(value=0, inplace=True)
a = StandardScaler().fit_transform(ratings_pivot)
print(a.shape)
np.set_printoptions(precision=2)
print(a[0:10, 0:10])

(3706, 6040)
[[ 9.81 -0.18 -0.11 -0.07 -0.22  7.18 -0.09  5.08  7.67  3.48]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09 -0.19 -0.17  3.48]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09 -0.19 -0.17 -0.34]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09  3.76 -0.17 -0.34]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09 -0.19 -0.17 -0.34]
 [-0.12 -0.18 -0.11 -0.07  2.43 -0.14  9.92 -0.19 -0.17 -0.34]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09 -0.19 -0.17  2.72]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09 -0.19 -0.17 -0.34]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09 -0.19 -0.17 -0.34]
 [-0.12 -0.18 -0.11 -0.07 -0.22 -0.14 -0.09 -0.19 -0.17 -0.34]]


In [154]:
print(type(ratings_pivot))
ratings_pivot.index.values

<class 'pandas.core.frame.DataFrame'>


array([   1,    2,    3, ..., 3950, 3951, 3952], dtype=int64)

3. Compute SVD to get U, S and V

In [18]:
U, Sigma, V = np.linalg.svd(a)

4. From your V.T select 50 components.

In [23]:
V_T = V[:50]

Shape of U ,S and V after reduction: 

In [120]:
print(U.shape)
print(Sigma.shape)
print(V.shape)
print(V_T.shape)

(3706, 3706)
(3706,)
(6040, 6040)
(50, 6040)


New, reduced matrix a we can recover by multiplying U ,S and V after reduction: 

In [26]:
a_reduced = V_T.dot(a.T)
a_reduced = a_reduced.T #we have to transpose in order to get movies as rows and components as columns
print(a_reduced.shape)

(3706, 50)


In order to have movie names, we have to import movies.dat file.

In [27]:
movies = pd.read_csv('movies.dat', sep="::", header=None, engine='python')
movies.columns = ['MovieID', 'Title', 'Genres']
movies.drop(['Genres'], axis=1, inplace=True)
print(movies[:5])

   MovieID                               Title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)


In [141]:
movies.loc[1, 'Title'] #the second row is the movie with id 2, but not necessarily

'Jumanji (1995)'

The function:

In [157]:
def recommend(movie_id, matrix):

# first we have to check if input is valid:
    if isinstance(movie_id, int) and isinstance(movie_id, np.int) and movie_id > 0 and movie_id <= matrix.shape[0]:
# initialize cos vector:
        cos = np.zeros(matrix.shape[0])
# find which row of a matrix is our movie:
        movie_loc = ratings_pivot.index.get_loc(movie_id)
# calculate cos sim between our movie and every other: 
        for i in range(0, matrix.shape[0]):
            a = np.dot(matrix[movie_loc], matrix[i])
            b = np.linalg.norm(matrix[movie_loc]) * np.linalg.norm(matrix[i])
            cos[i] = abs(a/b)

# sort in descending order:
        order = np.argsort(-cos)
# find ten best fits (the first one is going to be our movie): 
        ten = order[1:11]
        print('Recommended movies are:')
        print(ten)
        ten_most_similar = movies.loc[movies['MovieID'].isin(ten)]

    else:
        print('Invalid movie ID.')
    return(ten_most_similar)

Let us try with movie ID 2, 'Jumanji (1995)':

In [158]:
recommend(2, a_reduced)

Recommended movies are:
[3256  632   58 1980  308 1825  651 1787 3206 1873]


,MovieID,Title
57,58,"Postino, Il (The Postman) (1994)"
305,308,Three Colors: White (1994)
627,632,Land and Freedom (Tierra y libertad) (1995)
645,651,"Superweib, Das (1996)"
1728,1787,Paralyzing Fear: The Story of Polio in America...
1758,1825,"Player's Club, The (1998)"
1804,1873,"Misérables, Les (1998)"
1911,1980,Friday the 13th Part VII: The New Blood (1988)
3137,3206,Against All Odds (1984)
3187,3256,Patriot Games (1992)


6. Repeat the same process except now instead of using SVD you will use PCA to get the eigenvectors.
7. You will require co-variance matrix as an input to your eig function.

Matrix a is already standardized and NaNs are filled with 0. 

Eigenvectors and eigenvalues:

In [131]:
cov_mat = np.cov(a.T)
print(cov_mat.shape)

(6040, 6040)


In [132]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [53]:
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs_sorted = sorted(eig_pairs, key=lambda x: x[0])

eig_vec_reduced = np.stack([vec for val, vec in eig_pairs[:50]], axis=1)
print(eig_vec_reduced.shape)


(6040, 50)


8. Use that same steps after that to get 50 components and use cosine similarity to get the results.

In [54]:
a_reduced_PCA = np.matmul(a, eig_vec_reduced)
print(a.shape)
print(eig_vec_reduced.shape)
print(a_reduced_PCA.shape)

(3706, 6040)
(6040, 50)
(3706, 50)


In [159]:
recommend(2, a_reduced_PCA) 

Recommended movies are:
[3256  632   58 1980  308 1825  651 1787 3206 1873]


,MovieID,Title
57,58,"Postino, Il (The Postman) (1994)"
305,308,Three Colors: White (1994)
627,632,Land and Freedom (Tierra y libertad) (1995)
645,651,"Superweib, Das (1996)"
1728,1787,Paralyzing Fear: The Story of Polio in America...
1758,1825,"Player's Club, The (1998)"
1804,1873,"Misérables, Les (1998)"
1911,1980,Friday the 13th Part VII: The New Blood (1988)
3137,3206,Against All Odds (1984)
3187,3256,Patriot Games (1992)


9. Compare the results for SVD and PCA.

In [162]:
print('10 recommended movies for the movie with id 2, Jumanji (1995) using SVD:')
print(recommend(2, a_reduced))

10 recommended movies for the movie with id 2, Jumanji (1995) using SVD:
Recommended movies are:
[3256  632   58 1980  308 1825  651 1787 3206 1873]
      MovieID                                              Title
57         58                   Postino, Il (The Postman) (1994)
305       308                         Three Colors: White (1994)
627       632        Land and Freedom (Tierra y libertad) (1995)
645       651                              Superweib, Das (1996)
1728     1787  Paralyzing Fear: The Story of Polio in America...
1758     1825                          Player's Club, The (1998)
1804     1873                             Misérables, Les (1998)
1911     1980     Friday the 13th Part VII: The New Blood (1988)
3137     3206                            Against All Odds (1984)
3187     3256                               Patriot Games (1992)


In [125]:
print('10 recommended movies for the movie with id 2, Jumanji (1995) using PCA:')
print(recommend(2, a_reduced_PCA))

10 recommended movies for the movie with id 2, Jumanji (1995) using PCA:
[3256  632   58 1980  308 1825  651 1787 3206 1873]
      MovieID                                              Title
57         58                   Postino, Il (The Postman) (1994)
305       308                         Three Colors: White (1994)
627       632        Land and Freedom (Tierra y libertad) (1995)
645       651                              Superweib, Das (1996)
1728     1787  Paralyzing Fear: The Story of Polio in America...
1758     1825                          Player's Club, The (1998)
1804     1873                             Misérables, Les (1998)
1911     1980     Friday the 13th Part VII: The New Blood (1988)
3137     3206                            Against All Odds (1984)
3187     3256                               Patriot Games (1992)


As we can see, both lists are the same.